In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import art
from art.local import LocalBackend

model = art.TrainableModel(
    name="test",
    project="tests",
    base_model="Qwen/Qwen2.5-7B-Instruct",
    _internal_config=art.dev.InternalModelConfig(
        engine_args=art.dev.EngineArgs(
            num_scheduler_steps=1,
        ),
    ),
)
await model.register(LocalBackend())

wandb: Currently logged in as: bradhilton to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


INFO 08-22 02:21:51 [__init__.py:235] Automatically detected platform cuda.


/home/ubuntu/sky_workdir/src/art/__init__.py:10: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore # noqa: F401


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 08-22 02:21:59 [__init__.py:235] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.8.6: Fast Qwen2 patching. Transformers: 4.53.2. vLLM: 0.10.0.
   \\   /|    NVIDIA H100 PCIe. Num GPUs = 1. Max memory: 79.189 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-7b-instruct-unsloth-bnb-4bit with actual GPU utilization = 78.47%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 79.19 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill toke

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00, 25.05it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  2.53it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.29it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.39it/s]



INFO 08-22 02:22:23 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 08-22 02:22:24 [model_runner.py:1115] Model loading took 6.7355 GiB and 2.672879 seconds
INFO 08-22 02:22:27 [worker.py:295] Memory profiling takes 2.94 seconds
INFO 08-22 02:22:27 [worker.py:295] the current vLLM instance can use total_gpu_memory (79.19GiB) x gpu_memory_utilization (0.78) = 62.14GiB
INFO 08-22 02:22:27 [worker.py:295] model weights take 6.74GiB; non_torch_memory takes 0.14GiB; PyTorch activation peak memory takes 4.72GiB; the rest of the memory reserved for KV Cache is 50.54GiB.
INFO 08-22 02:22:27 [executor_base.py:113] # cuda blocks: 59142, # CPU blocks: 7021
INFO 08-22 02:22:27 [executor_base.py:118] Maximum concurrency for 32768 tokens per request: 28.88x
INFO 08-22 02:22:31 [vllm_utils.py:671] Unsloth: Running patched vLLM v0 `capture_model`.
INFO 08-22 02:22:31 [model_runner.py:1385] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To r

Capturing CUDA graph shapes: 100%|██████████| 49/49 [00:11<00:00,  4.12it/s]


INFO 08-22 02:22:43 [model_runner.py:1537] Graph capturing finished in 12 secs, took 1.35 GiB
INFO 08-22 02:22:43 [vllm_utils.py:678] Unsloth: Patched vLLM v0 graph capture finished in 12 secs.
INFO 08-22 02:22:43 [llm_engine.py:424] init engine (profile, create kv cache, warmup model) took 19.60 seconds
Unsloth: Just some info: will skip parsing ['k_norm', 'pre_feedforward_layernorm', 'post_feedforward_layernorm', 'q_norm']
Unsloth: Just some info: will skip parsing ['k_norm', 'pre_feedforward_layernorm', 'post_feedforward_layernorm', 'q_norm']


Unsloth 2025.8.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
client = model.openai_client()

chat_completion = await client.chat.completions.create(
    model="test",
    messages=[{"role": "user", "content": "Hi!"}],
)

In [7]:
from openai.types.chat.chat_completion_message_param import ChatCompletionMessageParam
from openai.types.chat.chat_completion_tool_param import ChatCompletionToolParam

message: ChatCompletionMessageParam = {"role": "user", "content": "Hi!"}
tools: list[ChatCompletionToolParam] = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }
]

chat_completion = await client.chat.completions.create(
    model="test",
    messages=[message],
    tool_choice="required",
    tools=tools,
    temperature=0.0,
)

In [17]:
from art.auto_trajectory import AutoTrajectoryContext

chunks = []
with AutoTrajectoryContext():
    async for chunk in await client.chat.completions.create(
        model="test",
        messages=[message],
        tool_choice="required",
        tools=tools,
        temperature=0.0,
        stream=True,
    ):
        chunks.append(chunk.model_dump(mode="json"))
    display(art.auto_trajectory().messages_and_choices[1].model_dump(mode="json"))
chunks

b'data: {"id":"chatcmpl-aad436fb6a564c3da716f2b9feb970a4","object":"chat.completion.chunk","created":1755831694,"model":"test","choices":[{"index":0,"delta":{"role":"assistant","content":""},"logprobs":null,"finish_reason":null}]}\n\ndata: {"id":"chatcmpl-aad436fb6a564c3da716f2b9feb970a4","object":"chat.completion.chunk","created":1755831694,"model":"test","choices":[{"index":0,"delta":{"role":"assistant","content":""},"logprobs":null,"finish_reason":null}]}\n\ndata: {"id":"chatcmpl-aad436fb6a564c3da716f2b9feb970a4","object":"chat.completion.chunk","created":1755831694,"model":"test","choices":[{"index":0,"delta":{"tool_calls":[{"id":"chatcmpl-tool-29e663261e524fcfa2162f4f3d76a7f0","type":"function","index":0,"function":{"name":"get_current_weather","arguments":"{"}}]},"logprobs":{"content":[{"token":"token_id:314","logprob":-0.00015293381875380874,"bytes":[32,123],"top_logprobs":[]}]},"finish_reason":null}]}\n\ndata: {"id":"chatcmpl-aad436fb6a564c3da716f2b9feb970a4","object":"chat.com

{'finish_reason': 'stop',
 'index': 0,
 'logprobs': {'content': [{'token': 'token_id:314',
    'bytes': [32, 123],
    'logprob': -0.00015293381875380874,
    'top_logprobs': []},
   {'token': 'token_id:314',
    'bytes': [32, 123],
    'logprob': -0.00015293381875380874,
    'top_logprobs': []},
   {'token': 'token_id:3417',
    'bytes': [125, 125],
    'logprob': -3.576278118089249e-07,
    'top_logprobs': []},
   {'token': 'token_id:3417',
    'bytes': [125, 125],
    'logprob': -3.576278118089249e-07,
    'top_logprobs': []}],
  'refusal': None},
 'message': {'content': None,
  'refusal': None,
  'role': 'assistant',
  'annotations': None,
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'chatcmpl-tool-29e663261e524fcfa2162f4f3d76a7f0',
    'function': {'arguments': '{{}}', 'name': 'get_current_weather'},
    'type': 'function'}]}}

[{'id': 'chatcmpl-aad436fb6a564c3da716f2b9feb970a4',
  'choices': [{'delta': {'content': '',
     'function_call': None,
     'refusal': None,
     'role': 'assistant',
     'tool_calls': None},
    'finish_reason': None,
    'index': 0,
    'logprobs': None}],
  'created': 1755831694,
  'model': 'test',
  'object': 'chat.completion.chunk',
  'service_tier': None,
  'system_fingerprint': None,
  'usage': None},
 {'id': 'chatcmpl-aad436fb6a564c3da716f2b9feb970a4',
  'choices': [{'delta': {'content': None,
     'function_call': None,
     'refusal': None,
     'role': None,
     'tool_calls': [{'index': 0,
       'id': 'chatcmpl-tool-29e663261e524fcfa2162f4f3d76a7f0',
       'function': {'arguments': '{', 'name': 'get_current_weather'},
       'type': 'function'}]},
    'finish_reason': None,
    'index': 0,
    'logprobs': {'content': [{'token': 'token_id:314',
       'bytes': [32, 123],
       'logprob': -0.00015293381875380874,
       'top_logprobs': []}],
     'refusal': None}}],
  '

In [8]:
chat_completion.model_dump(mode="json")

{'id': 'chatcmpl-e9aa2a86eed34c15b61cd3c70c54f268',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': {'content': [{'token': 'token_id:58',
      'bytes': [91],
      'logprob': -0.4935532212257385,
      'top_logprobs': []},
     {'token': 'token_id:4913',
      'bytes': [123, 34],
      'logprob': -0.0022107940167188644,
      'top_logprobs': []},
     {'token': 'token_id:606',
      'bytes': [110, 97, 109, 101],
      'logprob': 0.0,
      'top_logprobs': []},
     {'token': 'token_id:788',
      'bytes': [34, 58],
      'logprob': -2.0265558760002023e-06,
      'top_logprobs': []},
     {'token': 'token_id:330',
      'bytes': [32, 34],
      'logprob': 0.0,
      'top_logprobs': []},
     {'token': 'token_id:455',
      'bytes': [103, 101, 116],
      'logprob': -0.002185339340940118,
      'top_logprobs': []},
     {'token': 'token_id:11080',
      'bytes': [95, 99, 117, 114, 114, 101, 110, 116],
      'logprob': -3.576278118089249e-07,
      'top_logprobs': []

In [ ]:
chat_completion.model_dump(mode="json")